In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import numpy as np

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/")

X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

print ('Train data shape: ', X_train.shape)
print ('Train labels shape: ', y_train.shape)
print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', y_test.shape)

n_inputs = 28*28  # MNIST
n_outputs = 10

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Train data shape:  (55000, 784)
Train labels shape:  (55000,)
Test data shape:  (10000, 784)
Test labels shape:  (10000,)


In [3]:
logdir = './output'
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
for learning_rate in [10e-5, 10e-4, 10e-3, 10e-2, 10e-1, 1]:
    for batch_size in [32,64,128]:
        reset_graph()

        #X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        #y = tf.placeholder(tf.int64, shape=(None), name="y") 

        X = tf.placeholder(tf.float32, [None, n_inputs])
        y = tf.placeholder(tf.int64, [None])

        W = tf.Variable(tf.zeros([n_inputs, n_outputs]))
        b = tf.Variable(tf.zeros([n_outputs]))

        with tf.name_scope("dnn"):
            #logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
            logits = tf.nn.softmax(tf.matmul(X, W) + b)

        with tf.name_scope("loss"):
            xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
            loss = tf.reduce_mean(xentropy, name="loss")

#         learning_rate = 0.01

        with tf.name_scope("train"):
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            training_op = optimizer.minimize(loss)

        with tf.name_scope("eval"):
            correct = tf.nn.in_top_k(logits, y, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        with tf.name_scope("TrainAccuracy"):
            train_e_summary = tf.summary.scalar('lr_%.5f_B_%d'%(learning_rate,batch_size), accuracy)
        with tf.name_scope("TestAccuracy"):
            test_e_summary = tf.summary.scalar('lr_%.5f_B_%d'%(learning_rate,batch_size), accuracy)
        with tf.name_scope("Loss"):
            loss_summary = tf.summary.scalar('lr_%.5f_B_%d'%(learning_rate,batch_size), loss)


        init = tf.global_variables_initializer()
#         saver = tf.train.Saver()
        

        n_epochs = 20
        n_batches = mnist.train.num_examples // batch_size
        print(n_batches)

        with tf.Session() as sess:
            init.run()
            for epoch in range(n_epochs):

                for iteration in range(n_batches):
                    X_batch, y_batch = mnist.train.next_batch(batch_size)
                    sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

                    # training error every batch
                    summary_str = train_e_summary.eval(feed_dict={X: X_batch, y: y_batch})
                    step = epoch * n_batches + iteration
                    file_writer.add_summary(summary_str, step)

                # test error and loss every epoch
        #         acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #         epoch_loss = loss.eval(feed_dict={X: mnist.train.images, y: mnist.train.labels})
        #         acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
                summary_str = test_e_summary.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
                summary_str2 = loss_summary.eval(feed_dict={X: mnist.train.images, y: mnist.train.labels})
                step = epoch

                file_writer.add_summary(summary_str, step)
                file_writer.add_summary(summary_str2, step)
        #         print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test, "Avg Loss:", epoch_loss)

        #     save_path = saver.save(sess, "./my_model_final.ckpt")
        #     tf.merge_all_summaries() 
file_writer.close()

1718
859
429
1718
859
429
1718
859
429
1718
859
429
1718
859
429
1718
859
429


In [13]:
show_graph(tf.get_default_graph())

NameError: name 'show_graph' is not defined